# Concours sur les prix de vente immobiliers

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Lire les données
X = pd.read_csv('train.csv', index_col='Id') 
X_test = pd.read_csv('test.csv', index_col='Id')

# Supprimer les lignes avec des valeurs manquantes dans la cible, séparer la cible des prédicteurs
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

# Pour simplifier, nous allons supprimer les colonnes avec des valeurs manquantes
cols_with_missing = [col for col in X.columns if X[col].isnull().any()] 
X.drop(cols_with_missing, axis=1, inplace=True)
X_test.drop(cols_with_missing, axis=1, inplace=True)

# Séparer l'ensemble de validation de l'ensemble d'entraînement
X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                      train_size=0.8, test_size=0.2,
                                                      random_state=0)


## Enquête préleminaire

---

### 1. Etudes des valeurs manquantes sur les données numériques

Nous allons appliquer **l'imputation avec la médiane** pour gérer les valeurs manquantes dans un jeu de données. On utilise la fonction ***SimpleImputer de sklearn*** pour effectuer cette opération. Voici un exemple de code :

In [19]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

(1168, 60)


In [29]:
from sklearn.impute import SimpleImputer

# Sélectionner uniquement les colonnes numériques
numeric_cols = X_train.select_dtypes(include=['float64', 'int64']).columns

# Créer l'objet SimpleImputer avec la stratégie de la médiane
imputer = SimpleImputer(strategy='median')

# Appliquer l'imputation sur les colonnes numériques du jeu de données d'entraînement
X_train[numeric_cols] = imputer.fit_transform(X_train[numeric_cols])

# Appliquer l'imputation sur les colonnes numériques du jeu de données de validation
X_valid[numeric_cols] = imputer.transform(X_valid[numeric_cols])

# Vérification des valeurs manquantes après imputation
print("Nombre de valeurs manquantes dans X_train après imputation:")
print(X_train.isnull().sum())

print("\nNombre de valeurs manquantes dans X_valid après imputation:")
print(X_valid.isnull().sum())


Nombre de valeurs manquantes dans X_train après imputation:
MSSubClass       0
MSZoning         0
LotArea          0
Street           0
LotShape         0
LandContour      0
Utilities        0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
ExterQual        0
ExterCond        0
Foundation       0
BsmtFinSF1       0
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
Heating          0
HeatingQC        0
CentralAir       0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
BsmtFullBath     0
BsmtHalfBath     0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr     0
KitchenQual      0
TotRmsAbvGrd     0
Functional       0
Fireplaces       0
GarageCars       0
GarageArea       0
PavedDrive       0
WoodDeckS

### 2. Etudes des variables catégorielles
L'étude des variables catégorielles est une étape cruciale avant de les utiliser pour construire un modèle prédictif. 

---

#### 2.1. Identifier les variables catégorielles
Les variables catégorielles sont celles qui prennent des valeurs discrètes et peuvent être de type texte ou numérotées. Vous pouvez les identifier dans votre jeu de données en utilisant ***select_dtypes*** pour extraire les colonnes de type ***object ou category***.

In [36]:
# Identifier les variables catégorielles
categorical_cols = X_train.select_dtypes(include=['object']).columns
print(categorical_cols)

Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'Heating',
       'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'PavedDrive',
       'SaleType', 'SaleCondition'],
      dtype='object')


---
#### 2.2. Analyser la cardinalité
La cardinalité d'une variable catégorielle fait référence au nombre de valeurs uniques qu'elle peut prendre. Si une colonne a une grande cardinalité, l'encodage de cette variable (par exemple, via **l'encodage one-hot**) peut entraîner une explosion du nombre de caractéristiques, ce qui peut rendre la modélisation plus complexe.

On utilise ***nunique()*** pour obtenir la cardinalité des variables :

In [40]:
# Afficher la cardinalité des variables catégorielles
cardinality = X_train[categorical_cols].nunique()
print(cardinality)

MSZoning          5
Street            2
LotShape          4
LandContour       4
Utilities         2
LotConfig         5
LandSlope         3
Neighborhood     25
Condition1        9
Condition2        6
BldgType          5
HouseStyle        8
RoofStyle         6
RoofMatl          7
Exterior1st      15
Exterior2nd      16
ExterQual         4
ExterCond         5
Foundation        6
Heating           6
HeatingQC         5
CentralAir        2
KitchenQual       4
Functional        6
PavedDrive        3
SaleType          9
SaleCondition     6
dtype: int64


---
#### 2.2.1. Comparer les valeurs uniques dans les colonnes catégorielles
Pour chaque colonne catégorielle, nous allons comparer les valeurs uniques dans l'ensemble d'entraînement (X_train) et l'ensemble de validation (X_valid).

Voici le code pour le faire :

In [46]:
# Comparer les valeurs uniques des variables catégorielles entre l'ensemble d'entraînement et l'ensemble de validation
for col in categorical_cols:
    train_unique = set(X_train[col].unique())
    valid_unique = set(X_valid[col].unique())
    
    # Trouver les différences
    train_only = train_unique - valid_unique
    valid_only = valid_unique - train_unique
    
    if train_only or valid_only:
        print(f"Colonne: {col}")
        print(f"Valeurs uniques uniquement dans l'entraînement: {train_only}")
        print(f"Valeurs uniques uniquement dans la validation: {valid_only}")
        print("-" * 50)

Colonne: Utilities
Valeurs uniques uniquement dans l'entraînement: {'NoSeWa'}
Valeurs uniques uniquement dans la validation: set()
--------------------------------------------------
Colonne: LandSlope
Valeurs uniques uniquement dans l'entraînement: {'Sev'}
Valeurs uniques uniquement dans la validation: set()
--------------------------------------------------
Colonne: Neighborhood
Valeurs uniques uniquement dans l'entraînement: {'Blueste'}
Valeurs uniques uniquement dans la validation: set()
--------------------------------------------------
Colonne: Condition1
Valeurs uniques uniquement dans l'entraînement: {'RRNe'}
Valeurs uniques uniquement dans la validation: set()
--------------------------------------------------
Colonne: Condition2
Valeurs uniques uniquement dans l'entraînement: {'RRAe', 'PosA'}
Valeurs uniques uniquement dans la validation: {'RRNn', 'RRAn'}
--------------------------------------------------
Colonne: RoofStyle
Valeurs uniques uniquement dans l'entraînement: {'She

**Problèmes potentiels** : Certaines colonnes de l'ensemble d'entraînement contiennent des valeurs uniques absentes dans l'ensemble de validation. Par exemple, les colonnes Utilities, LandSlope, Neighborhood, etc., contiennent des valeurs uniques qui n'apparaissent pas dans l'ensemble de validation.

---

#### 2.2.2. Ajout de la catégorie "UNKNOWN"

In [51]:
import pandas as pd

# Remplacer les valeurs uniques présentes dans l'entraînement mais pas dans la validation par "UNKNOWN"
for col in X_train.select_dtypes(include=['object']).columns:
    train_unique = set(X_train[col].unique())
    valid_unique = set(X_valid[col].unique())
    
    # Valeurs uniquement présentes dans l'entraînement
    unique_in_train = train_unique - valid_unique
    
    # Remplacer les valeurs uniques dans l'entraînement par "UNKNOWN"
    if unique_in_train:
        X_train[col] = X_train[col].replace(unique_in_train, 'UNKNOWN')
    
    # Valeurs uniquement présentes dans la validation
    unique_in_valid = valid_unique - train_unique
    
    # Remplacer les valeurs uniques dans la validation par "UNKNOWN"
    if unique_in_valid:
        X_valid[col] = X_valid[col].replace(unique_in_valid, 'UNKNOWN')

# Vérifier les valeurs uniques après modification
for col in X_train.select_dtypes(include=['object']).columns:
    print(f"Colonne: {col}")
    print("Valeurs uniques dans l'entraînement:", set(X_train[col].unique()))
    print("Valeurs uniques dans la validation:", set(X_valid[col].unique()))
    print("-" * 50)


Colonne: MSZoning
Valeurs uniques dans l'entraînement: {'RL', 'C (all)', 'RM', 'FV', 'RH'}
Valeurs uniques dans la validation: {'RL', 'C (all)', 'RM', 'FV', 'RH'}
--------------------------------------------------
Colonne: Street
Valeurs uniques dans l'entraînement: {'Pave', 'Grvl'}
Valeurs uniques dans la validation: {'Pave', 'Grvl'}
--------------------------------------------------
Colonne: LotShape
Valeurs uniques dans l'entraînement: {'Reg', 'IR1', 'IR2', 'IR3'}
Valeurs uniques dans la validation: {'Reg', 'IR1', 'IR2', 'IR3'}
--------------------------------------------------
Colonne: LandContour
Valeurs uniques dans l'entraînement: {'Lvl', 'Low', 'Bnk', 'HLS'}
Valeurs uniques dans la validation: {'Lvl', 'Low', 'Bnk', 'HLS'}
--------------------------------------------------
Colonne: Utilities
Valeurs uniques dans l'entraînement: {'UNKNOWN', 'AllPub'}
Valeurs uniques dans la validation: {'AllPub'}
--------------------------------------------------
Colonne: LotConfig
Valeurs unique

---

## 3. Encodage One Hot
Nous utiliserons l'encodage One-Hot pour transformer ces variables en variables binaires.

In [58]:
from sklearn.preprocessing import OneHotEncoder

# Sélectionner les colonnes catégorielles
categorical_cols = X_train.select_dtypes(include=['object']).columns

# Initialiser l'encodeur OneHot
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Appliquer l'encodage sur les colonnes catégorielles de l'entraînement et de la validation
X_train_encoded = encoder.fit_transform(X_train[categorical_cols])
X_valid_encoded = encoder.transform(X_valid[categorical_cols])

# Convertir les résultats encodés en DataFrame avec les noms des colonnes appropriés
encoded_columns = encoder.get_feature_names_out(categorical_cols)
X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=encoded_columns, index=X_train.index)
X_valid_encoded_df = pd.DataFrame(X_valid_encoded, columns=encoded_columns, index=X_valid.index)

# Ajouter les colonnes encodées aux autres variables non catégorielles
X_train_final = pd.concat([X_train.drop(categorical_cols, axis=1), X_train_encoded_df], axis=1)
X_valid_final = pd.concat([X_valid.drop(categorical_cols, axis=1), X_valid_encoded_df], axis=1)

# Vérifier les premières lignes du jeu de données final
print(X_train_final.head())
print(X_valid_final.head())

     MSSubClass  LotArea  OverallQual  OverallCond  YearBuilt  YearRemodAdd  \
Id                                                                            
619        20.0  11694.0          9.0          5.0     2007.0        2007.0   
871        20.0   6600.0          5.0          5.0     1962.0        1962.0   
93         30.0  13360.0          5.0          7.0     1921.0        2006.0   
818        20.0  13265.0          8.0          5.0     2002.0        2002.0   
303        20.0  13704.0          7.0          5.0     2001.0        2002.0   

     BsmtFinSF1  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  ...  SaleType_New  \
Id                                                   ...                 
619        48.0         0.0     1774.0       1822.0  ...           1.0   
871         0.0         0.0      894.0        894.0  ...           0.0   
93        713.0         0.0      163.0        876.0  ...           0.0   
818      1218.0         0.0      350.0       1568.0  ...           0.0   
30

**Vérifier les colonnes non numériques** : Identifiez les colonnes qui ne sont pas numériques dans vos ensembles de données.

In [71]:
non_numeric_columns_train = X_train.select_dtypes(include=['object']).columns
non_numeric_columns_valid = X_valid.select_dtypes(include=['object']).columns

print("Colonnes non numériques dans X_train:", non_numeric_columns_train)
print("Colonnes non numériques dans X_valid:", non_numeric_columns_valid)


Colonnes non numériques dans X_train: Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'Heating',
       'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'PavedDrive',
       'SaleType', 'SaleCondition'],
      dtype='object')
Colonnes non numériques dans X_valid: Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'Heating',
       'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'PavedDrive',
       'SaleType', 'SaleCondition'],
      dtype='object')


In [73]:
# Refaire un encodage one-hot cohérent
full_data = pd.concat([X_train, X_valid], keys=['train', 'valid'])
full_data_encoded = pd.get_dummies(full_data)

# Séparer les ensembles après encodage
X_train = full_data_encoded.loc['train']
X_valid = full_data_encoded.loc['valid']


In [75]:
train_columns = set(X_train.columns)
valid_columns = set(X_valid.columns)
missing_in_train = valid_columns - train_columns
missing_in_valid = train_columns - valid_columns

print("Colonnes manquantes dans X_train:", missing_in_train)
print("Colonnes manquantes dans X_valid:", missing_in_valid)


Colonnes manquantes dans X_train: set()
Colonnes manquantes dans X_valid: set()


**Ajoutez les colonnes manquantes avec des valeurs par défaut (0)** :

In [80]:
for col in missing_in_train:
    X_train[col] = 0
for col in missing_in_valid:
    X_valid[col] = 0

# Réaligner les colonnes
X_train = X_train[X_valid.columns]


---

## 4. Evaluation du modèle

Voici comment procéder avec Random Forest et en utilisant MAE (Mean Absolute Error) comme métrique d'évaluation. Nous allons suivre un processus étape par étape :

In [82]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train, X_valid, y_train, y_valid, n_estimators=100, max_depth=None, random_state=42):
    """
    Évalue un modèle Random Forest avec les données fournies et retourne le MAE.

    Parameters:
    - X_train, X_valid: features d'entraînement et de validation
    - y_train, y_valid: cibles d'entraînement et de validation
    - n_estimators: nombre d'arbres dans la forêt
    - max_depth: profondeur maximale des arbres
    - random_state: graine aléatoire pour la reproductibilité

    Returns:
    - Mean Absolute Error (MAE)
    """
    # Initialiser le modèle
    model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state)
    
    # Entraîner le modèle
    model.fit(X_train, y_train)
    
    # Faire des prédictions
    predictions = model.predict(X_valid)
    
    # Calculer et retourner le MAE
    return mean_absolute_error(y_valid, predictions)


In [84]:
mae = score_dataset(X_train, X_valid, y_train, y_valid, n_estimators=100)
print(f"MAE for Random Forest: {mae}")

MAE for Random Forest: 17586.36378995434


Le Mean Absolute Error (MAE) obtenu pour le modèle Random Forest est de 17,586.36. Cela signifie que, en moyenne, le modèle fait une erreur de **17,586.36 unités monétaires** (peut-être dollars ou une autre devise) pour prédire les prix de vente.

----

## 4. Faire des prédictions sur les données de test
Utilisez le modèle pour prédire les prix des maisons sur le fichier de test.


**Identification des colonnes catégorielles à faible cardinalité**

In [105]:
# Identifier les colonnes catégorielles (type object ou catégorie)
categorical_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']

# Sélectionner les colonnes à faible cardinalité (moins de 10 catégories uniques, par exemple)
low_cardinality_cols = [col for col in categorical_cols if X_train[col].nunique() < 10]

# Vérifier la liste des colonnes identifiées
print("Colonnes catégorielles à faible cardinalité :", low_cardinality_cols)


Colonnes catégorielles à faible cardinalité : []


In [107]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Initialiser l'encodeur
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Entraîner l'encodeur sur X_train
encoder.fit(X_train[low_cardinality_cols])

# Transformer X_train et X_test
X_train_encoded = encoder.transform(X_train[low_cardinality_cols])
X_test_encoded = encoder.transform(X_test[low_cardinality_cols])

# Obtenir les noms des colonnes encodées
encoded_columns = encoder.get_feature_names_out(low_cardinality_cols)

# Convertir en DataFrame
X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=encoded_columns, index=X_train.index)
X_test_encoded_df = pd.DataFrame(X_test_encoded, columns=encoded_columns, index=X_test.index)


**Alignement des colonnes entre les jeux de données d'entraînement et de test**

In [115]:
# Convertir X_train_encoded en DataFrame
X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=encoder.get_feature_names_out(low_cardinality_cols), index=X_train.index)

# Convertir X_test_encoded en DataFrame
X_test_encoded_df = pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names_out(low_cardinality_cols), index=X_test.index)


In [117]:
# Réaligner les colonnes de X_test_encoded_df sur celles de X_train_encoded_df
X_test_encoded_df = X_test_encoded_df.reindex(columns=X_train_encoded_df.columns, fill_value=0)


In [119]:
# Sélectionner les colonnes numériques
numerical_cols = [col for col in X_train.columns if X_train[col].dtype in ['int64', 'float64']]

# Combiner les colonnes numériques et encodées
X_train_final = pd.concat([X_train[numerical_cols], X_train_encoded_df], axis=1)
X_test_final = pd.concat([X_test[numerical_cols], X_test_encoded_df], axis=1)

# Vérifier les dimensions finales
print("Dimensions de X_train_final :", X_train_final.shape)
print("Dimensions de X_test_final :", X_test_final.shape)


Dimensions de X_train_final : (1168, 33)
Dimensions de X_test_final : (1459, 33)


**Prédictions sur le jeu de test**

In [167]:
from sklearn.ensemble import RandomForestRegressor

# Créer et entraîner le modèle
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_final, y_train)  # X_train_final et y_train doivent être correctement définis

# Faire des prédictions sur le jeu de test
predictions = model.predict(X_test_final)

# Convertir les prédictions en DataFrame
predictions_df = pd.DataFrame(predictions, columns=['SalePrice'])

# Afficher les premières lignes
predictions_df.head()

# Récupérer les IDs de l'index
X_test_ids = X_test.index

# Créer un DataFrame pour soumettre les prédictions
output = pd.DataFrame({'Id': X_test_ids, 'SalePrice': predictions})
output.to_csv('submission3.csv', index=False)
